# Búsquedas semánticas con películas

## Entorno:

```shellscript
docker run -p 6333:6333 -p 6334:6334 -v $(pwd)/q_storage:/qdrant/storage:z qdrant/qdrant
```

```shellscript	
python -m venv .venv
source .venv/bin/activate
```


In [ ]:
%pip install -r requirements.txt
#!jupyter nbextension enable --py widgetsnbextension


In [ ]:
# Ejemplo con Chroma DB para insertar y consultar
import chromadb

# Inicialización del cliente Chroma
client = chromadb.Client()

# Creación de una colección
try:
    client.delete_collection("basic_example")
except:
    pass
# Chroma Distance metric: https://docs.trychroma.com/usage-guide#changing-the-distance-function
collection = client.create_collection(name="basic_example", metadata={"hnsw:space": "l2"})

# Creación de 4 vectores de dimensión 3
vectores = [
    [0.1, 0.1, 0.1],
    [0.2, 0.2, 0.2],
    [0.3, 0.3, 0.3],
    [0.4, 0.4, 0.4]
]
metadatas = [
    {"color": "blue"},
    {"color": "blue"},
    {"color": "red"},
    {"color": "red"}
]

collection.upsert(ids=["1", "2", "3", "4"], embeddings=vectores, metadatas=metadatas)

q_embedding = [0.12, 0.12, 0.12]
results = collection.query(query_embeddings=q_embedding, n_results=3, include = ["metadatas", "embeddings", "distances"],)
print(f'Resultados para embedding: {q_embedding}')
for i, id in enumerate(results['ids'][0]):
    distance = results['distances'][0][i]
    metadata = results['metadatas'][0][i]
    vector = results['embeddings'][0][i]
    print(f' - ID: {id}, Distance: {distance:.5f}, Metadata: {metadata}, Vector: [{vector[0]:.1f}, ...]')

results = collection.query(query_embeddings=q_embedding, where={"color": "red"}, n_results=3, include = ["metadatas", "embeddings", "distances"],)
print(f'Resultados para embedding: {q_embedding} con filtro color=red')
for i, id in enumerate(results['ids'][0]):
    distance = results['distances'][0][i]
    metadata = results['metadatas'][0][i]
    vector = results['embeddings'][0][i]
    print(f' - ID: {id}, Distance: {distance:.5f}, Metadata: {metadata}, Vector: [{vector[0]:.1f}, ...]')



In [22]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_json('datasets/star_wars_plots.json')
print('Columns:', df.columns, ', Records:', df.shape[0])
df.head()
startwars_movies = df

Columns: Index(['title', 'plot'], dtype='object') , Records: 11


### Creamos embeddings del argumento de las películas

In [23]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-minilm-l6-v2', device='cuda')
#model = SentenceTransformer('all-mpnet-base-v2', device='cuda')
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
)


### Preparamos los datos para almacenarlos en la BBDD Vectorial

In [35]:
import time
from tqdm.notebook import tqdm

CHUNK_WORDS = 200

# Creamos chunks de texto de unas 200 palabras con un solape de 10 palabras
def chunk_text(text, chunk_size=CHUNK_WORDS, overlap=CHUNK_WORDS//20):
    '''
    Divide el texto en chunks de un tamaño dado con un solape entre ellos.
    Cada modelo solo puede procesar un número limitado de tokens, por lo que 
    es necesario dividir el texto en chunks.
    1000 tokens son aproximadamente 750 palabras.
    https://openai.com/pricing#language-models
    '''
    chunks = []
    words = text.split()
    for i in range(0, len(words), chunk_size-overlap):
        chunks.append(' '.join(words[i:i+chunk_size]))
    return chunks

# Añadimos columnas plot_chunks y chunk_embeddings que serán arrays en  startwars_movies
plot_embeddings = []
for i, row in tqdm(startwars_movies.iterrows()):
    plot_chunks = chunk_text(row['plot'])
    embeddings = model.encode(plot_chunks).tolist()
    plot_embeddings.append([{'chunk': chunk, 'embedding': embedding} for chunk, embedding in zip(plot_chunks, embeddings)])
    print(i, row['title'], 'chunks:', len(plot_chunks))
    


0it [00:00, ?it/s]

0 Star Wars: Episodio IV - Una nueva esperanza chunks: 8
1 Star Wars: Episodio V - El Imperio contraataca chunks: 12
2 Star Wars: Episode VI - Return of the Jedi chunks: 17
3 Star Wars: Episodio I - La amenaza fantasma chunks: 5
4 Star Wars: Episodio II - El ataque de los clones chunks: 13
5 Star Wars: Episodio III - La venganza de los Sith chunks: 28
6 Star Wars: Episodio VII - El despertar de la Fuerza chunks: 20
7 Star Wars: Episodio VIII - Los últimos Jedi chunks: 16
8 Star Wars: Episodio IX - El ascenso de Skywalker chunks: 17
9 Rogue One: una historia de Star Wars chunks: 20
10 Han Solo: una historia de Star Wars chunks: 6


In [36]:
#Creamos un documento por cada chunk de texto de cada película con su embedding y metadatos

vector_data = []
i = 0
for k, title in enumerate(startwars_movies['title'].tolist()):
    #print(md)
    for plot_chunk in plot_embeddings[k]:
        metadata = dict(title=title)
        i += 1
        metadata['plot_chunk'] = plot_chunk['chunk'] 
        vector_data.append({
            "id": i,
            "embedding": plot_chunk['embedding'],
            "metadata": metadata,
        })

for v in vector_data[:10]:
    print(v['id'], v['metadata']['title'],'-', v['metadata']['plot_chunk'][:80])
print('Vectors:', len(vector_data), 'Movies:', len(startwars_movies))  


1 Star Wars: Episodio IV - Una nueva esperanza - Hace mucho tiempo en una galaxia muy, muy lejana ... Son tiempos de guerra civil
2 Star Wars: Episodio IV - Una nueva esperanza - Darth Vader a la cabeza. Durante el asalto, capturan a la princesa, quien antes 
3 Star Wars: Episodio IV - Una nueva esperanza - travesía, son atacados por «moradores de las arenas», siendo finalmente rescatad
4 Star Wars: Episodio IV - Una nueva esperanza - todos sus habitantes. Al regresar a su casa, Luke encuentra los cadáveres carbon
5 Star Wars: Episodio IV - Una nueva esperanza - de que no colabore con el imperio. Debido a esto, Leia cede y le dice a Tarkin q
6 Star Wars: Episodio IV - Una nueva esperanza - contrabando. Después, descubren que la princesa se encuentra en la estación, por
7 Star Wars: Episodio IV - Una nueva esperanza - no está del todo convencida, pues según su criterio los han dejado escapar así p
8 Star Wars: Episodio IV - Una nueva esperanza - rebelde del Templo Massassi, donde la pri

### Conectamos con Qdrant para crear una colección y alimentarla con los embeddings


In [37]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

qd = QdrantClient(url="http://localhost:6333")

MOVIES_COLLECTION = "movies_sw"

qd.delete_collection(MOVIES_COLLECTION)
# "size" is the dimension of the vectors, "distance" is the metric used to calculate the distance between vectors
qd.create_collection(MOVIES_COLLECTION, VectorParams(size=model.get_sentence_embedding_dimension(), distance=Distance.COSINE))
movies = qd.get_collection(MOVIES_COLLECTION)

print(movies)

status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=0 indexed_vectors_count=0 points_count=0 segments_count=8 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None) payload_schema={}


In [38]:
from qdrant_client.models import PointStruct

points = list(map(lambda x: PointStruct(id=int(x['id']), vector=x['embedding'], payload=x['metadata']), vector_data))
print('Points:', len(points), 'Movies:', len(startwars_movies))
op = qd.upsert(
    collection_name=MOVIES_COLLECTION,
    wait=True,
    points=points,
)

print(qd.get_collection(MOVIES_COLLECTION))



Points: 162 Movies: 11
status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=162 indexed_vectors_count=0 points_count=162 segments_count=8 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None) payload_schema={}


### Realizamos búsquedas semánticas sobre la saga Star Wars (Spoilers warning!!)

In [39]:


queries = [
    'Anakin gana una carrera cuando era un niño',
    'Los clones reciben la orden de ejecutar a los Jedi',
    'Han Solo gana el Halcón Milenario en una partida de cartas',
    #'Kylo Ren mata a su padre, Han Solo',
    #'Palpatine es derrotado definitivamente por Rey',
    # Less accurate queries
    'Luke descubre que Darth Vader es en realidad su padre',
    'Luke encuentra a Yoda y es entrenado como Jedi',
]

for q in queries:
    query_emb = model.encode(q)
    results = qd.search(MOVIES_COLLECTION, query_vector=query_emb, limit=3)
    print('Q:', q)
    for r in results:
        print('     ', r.score, ' => ', r.payload['title'])
        print('     ', f'(id: {r.id})', r.payload['plot_chunk'])



Q: Anakin gana una carrera cuando era un niño
      0.6354809  =>  Star Wars: Episodio I - La amenaza fantasma
      (id: 41) el niño gana la carrera, se quedarán con los repuestos para la nave y con la libertad de Anakin. Watto acepta, al creer que el mayor oponente de Anakin, Sebulba, le ganaría.Ya en la carrera, Anakin tiene problemas ya que Sebulba había roto un elemento de su pod. Sin embargo, el niño se las ingenia para estabilizar la nave y finalmente gana la carrera. Luego de una emotiva despedida entre Shmi y Anakin, todos se dirigen hacia la nave de la Reina para trasladarse hacia Coruscant, capital de la República. Antes de llegar a la nave, Qui-Gon es sorprendido por el aprendiz de Lord Sidious, Darth Maul, y mantienen una pequeña lucha con sus sables. Finalmente, Qui-Gon logra huir en la nave. Una vez en Coruscant, Qui-Gon y Obi-Wan se dirigen hacia el Consejo Jedi para proponer el entrenamiento de Anakin, pero es rechazado debido a que el consejo se opuso a ello porque no